In [5]:
#encoding=utf-8
import pandas as pd
import numpy as np
import re
import string

In [3]:
path = '../../TimePattern/time_words'

def get_time_words(path):
    with open(path, encoding='utf-8') as f:
        time_words = set()
        for each in f:
            each = each.strip('\n')
            # print(each,end='')
            time_words.add(each)
            pattern_words = '|'.join(time_words)
        return pattern_words
pattern = get_time_words(path)

In [8]:
# re.findall(pattern, '我明天还或者下午还')
re.sub(pattern,'','我明天还或者下午还')

'我还或者还'

In [3]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../../data/CutDebt/'
# data = pd.read_csv(path + 'mock_up_data.csv', encoding='utf8')
data = pd.read_csv(path + 'mock_up_data1.csv', encoding='utf8')

# data = data.drop(['id','text','备注'], axis = 1)
data = data.dropna()
data.head(10)

,label,split_text
0,0,可以
1,1,我 现在 实在 没 那么 多 钱
2,2,今天 天气 不错
3,0,行
4,0,好
5,0,没问题 那 我 在 哪天 之前 需要 还清 呢
6,0,这样 我 就 比较 可以 接受
7,0,嗯嗯 这个 金额 可以的
8,0,好的 这个 金额 我还能 接受
9,0,这个 我 就 可以还 了


# use goslate to get more data

In [4]:
from textblob import TextBlob
from textblob.translate import NotTranslated
import pandas as pd
import numpy as np
from multiprocessing import Pool
from itertools import repeat

In [5]:
def translate(comment, from_lang, to_lang):
        if hasattr(comment, "decode"):
            comment = comment.decode("utf-8")

        text = TextBlob(comment)
        try:
            text = text.translate(to=to_lang)
            text = text.translate(to=from_lang)
        except NotTranslated:
            pass

        return str(text)

def translate_csv(df,col,from_lang,to_lang,num_pol=10):
    """
        https://developers.google.com/translate/v2/using_rest#language-params
    """

    df = df.copy()

    if num_pol >= 1000:
        num_pol=1000
    comment_pool = df[col].values
    p = Pool(num_pol)
    new_col_name = col + '_' + to_lang
    df[new_col_name] = p.starmap(translate, zip(comment_pool, repeat(from_lang),repeat(to_lang)))
    df = df.drop([col], axis = 1)
    df = df.rename(index=str, columns={new_col_name:col})
    return df

In [6]:
col = 'split_text'
data_en = translate_csv(data,col,from_lang='zh',to_lang='en',num_pol=50)
print('finish 1st trans')
data_fr = translate_csv(data,col,from_lang='zh',to_lang='fr',num_pol=50)
print('finish 2nd trans')
data_th = translate_csv(data,col,from_lang='zh',to_lang='th',num_pol=50)
print('finish 3rd trans')
data_lo = translate_csv(data,col,from_lang='zh',to_lang='lo',num_pol=50)
print('finish 4th trans')

finish 1st trans
finish 2nd trans
finish 3rd trans
finish 4th trans


In [7]:
data = pd.concat([data,data_en,data_fr,data_th,data_lo], ignore_index=True)
data

,label,split_text
0,0,可以
1,1,我 现在 实在 没 那么 多 钱
2,2,今天 天气 不错
3,0,行
4,0,好
5,0,没问题 那 我 在 哪天 之前 需要 还清 呢
6,0,这样 我 就 比较 可以 接受
7,0,嗯嗯 这个 金额 可以的
8,0,好的 这个 金额 我还能 接受
9,0,这个 我 就 可以还 了


In [8]:
import jieba
jieba.load_userdict("../../WordCut/userdict.txt")
# use jieba to cut words

def cut_words(text):
    seg_list = jieba.cut(text, cut_all=False)
    return " ".join(seg_list)

data['split_text']=data['split_text'].apply(cut_words)
data

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.490 seconds.
Prefix dict has been built succesfully.


,label,split_text
0,0,可以
1,1,我 现在 实在 没 那么 多 钱
2,2,今天 天气 不错
3,0,行
4,0,好
5,0,没 问题 那 我 在 哪 天 之前 需要 还清 呢
6,0,这样 我 就 比较 可以 接受
7,0,嗯 嗯 这个 金额 可以 的
8,0,好 的 这个 金额 我 还 能 接受
9,0,这个 我 就 可以 还 了


# data cleaning and save

In [9]:
def clean(text):
    text = re.sub(f'([{string.punctuation}“”¨«»®´·º ½¾¿¡§£₤‘’，])',' ', text)
    text = text.split(' ')
    text = ' '.join(text)
    return text

def clean_label(label):
    return int(label)
    
print('split_text cleaning')
# data['split_text'].head(10)
data['split_text'] = data['split_text'].apply(clean)
data['label'] = data['label'].apply(clean_label)

# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

data.to_csv(path + 'cleaned_mock_up_data.csv', index = False, encoding = 'utf8')

split_text cleaning


In [10]:
data.label.values

array([2, 1, 2, ..., 0, 2, 1])